In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

df = pd.read_csv("./Data/emotion.tsv", sep="\t")

print(df.columns)

null_idx = df[df["document"].isnull()].index
df.loc[null_idx]

train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

print("중복 제거 전 학습 데이터셋: {}".format(len(train_data)))
print("중복 제거 전 테스트 데이터셋: {}".format(len(test_data)))

train_data = train_data.drop_duplicates(["document"])
test_data = test_data.drop_duplicates(["document"])

print("중복 제거 후 학습 데이터셋: {}".format(len(train_data)))
print("중복 제거 후 테스트 데이터셋: {}".format(len(test_data)))

MODEL_NAME = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_train_sentences = tokenizer(
    list(train_data['document']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True,
)

tokenizer_test_sentences = tokenizer(
    list(test_data['document']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True,
)


class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
train_label = train_data['label'].values
test_label = test_data['label'].values

train_dataset = CurseDataset(tokenizer_train_sentences, train_label)
test_dataset = CurseDataset(tokenizer_test_sentences, test_label)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to(device)

traning_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="wegihted")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
}

trainer = Trainer(
    model=model,
    args=traning_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate(eval_dataset=test_dataset)

model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

Index(['document', 'label'], dtype='object')
중복 제거 전 학습 데이터셋: 240525
중복 제거 전 테스트 데이터셋: 60131
중복 제거 후 학습 데이터셋: 234389
중복 제거 후 테스트 데이터셋: 59363


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/21975 [00:00<?, ?it/s]

C:\Users\jhy97\AppData\Local\Temp\ipykernel_53292\824084575.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 0.6037, 'grad_norm': 7.523480415344238, 'learning_rate': 5e-05, 'epoch': 0.07}


...

{'loss': 0.0835, 'grad_norm': 3.3764729499816895, 'learning_rate': 1.1059371362048895e-06, 'epoch': 2.94}


C:\Users\jhy97\AppData\Local\Temp\ipykernel_53292\824084575.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'train_runtime': 7327.1291, 'train_samples_per_second': 95.968, 'train_steps_per_second': 2.999, 'train_loss': 0.17812271031367766, 'epoch': 3.0}


  0%|          | 0/1856 [00:00<?, ?it/s]

InvalidParameterError: The 'average' parameter of precision_recall_fscore_support must be a str among {'binary', 'micro', 'weighted', 'samples', 'macro'} or None. Got 'wegihted' instead.

In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# gpu 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:

# 저장된 모델과 토크나이저 불러오기
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def sentence_predict(sent):
    model.eval()
    tokenized_sent = tokenizer(
        [sent],
        return_tensors="pt",
        max_length=128,
        padding=True,
        truncation=True,
        add_special_tokens=True,
    )
    tokenized_sent = {k: v.to(device) for k, v in tokenized_sent.items()}
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
        )
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1).numpy()[0]
    return result

# 데이터 로드 및 샘플링
data = pd.read_csv("Data/emotion.tsv", sep='\t')
sampled_data = data.sample(n=1000)

# 정확도 계산을 위한 변수 초기화
correct_predictions = 0

for index, row in sampled_data.iterrows():
    sentence = row['document']
    true_label = row['label']
    predicted_label = sentence_predict(sentence)
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}")
    if true_label == predicted_label:
        correct_predictions += 1

accuracy = correct_predictions / len(sampled_data)
print(f"정확도: {accuracy * 100:.2f}%")

True Label: 2, Predicted Label: 2
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 1, Predicted Label: 1
True Label: 1, Predicted Label: 1
True Label: 2, Predicted Label: 2
True Label: 2, Predicted Label: 2
True Label: 2, Predicted Label: 2
True Label: 1, Predicted Label: 1
True Label: 1, Predicted Label: 1
True Label: 2, Predicted Label: 2
True Label: 2, Predicted Label: 2
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 0, Predicted Label: 0
True Label: 0, Predicted Label: 0
True Label: 2, Predicted Label: 2
True Label: 0, Predicted Label: 0
True Label: 0, Predicted Label: 0
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 0, Predicted Label: 0
True Label: 0, Predicted Label: 0
True Label: 1, Predicted Label: 1
True Label: 2, Predicted Label: 2
True Label: 1,

In [5]:

# 저장된 모델과 토크나이저 불러오기
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def sentence_predict(sent):
    model.eval()
    tokenized_sent = tokenizer(
        [sent],
        return_tensors="pt",
        max_length=128,
        padding=True,
        truncation=True,
        add_special_tokens=True,
    )
    tokenized_sent = {k: v.to(device) for k, v in tokenized_sent.items()}
    with torch.no_grad():
        outputs = model(**tokenized_sent)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_label = torch.argmax(probabilities, dim=1).item()
    return predicted_label, probabilities.tolist()[0]

# 데이터 로드 및 샘플링
data = pd.read_csv("Data/emotion.tsv", sep='\t')
sampled_data = data.sample(n=1000)

# 정확도 계산을 위한 변수 초기화
correct_predictions = 0

for index, row in sampled_data.iterrows():
    sentence = row['document']
    true_label = row['label']
    predicted_label, probabilities = sentence_predict(sentence)
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}, Probabilities: {probabilities}")
    if true_label == predicted_label:
        correct_predictions += 1

accuracy = correct_predictions / len(sampled_data)
print(f"정확도: {accuracy * 100:.2f}%")

True Label: 0, Predicted Label: 0, Probabilities: [0.9993261098861694, 0.0005724413204006851, 0.00010144706902792677]
True Label: 2, Predicted Label: 2, Probabilities: [0.011039125733077526, 0.004734841175377369, 0.9842260479927063]
True Label: 1, Predicted Label: 1, Probabilities: [0.0011056532384827733, 0.9988265633583069, 6.773103086743504e-05]
True Label: 1, Predicted Label: 1, Probabilities: [0.015341976657509804, 0.9841603636741638, 0.0004976809723302722]
True Label: 0, Predicted Label: 0, Probabilities: [0.9993076324462891, 0.0005849307053722441, 0.00010743358143372461]
True Label: 1, Predicted Label: 1, Probabilities: [0.000939542253036052, 0.9990046620368958, 5.581815275945701e-05]
True Label: 2, Predicted Label: 2, Probabilities: [4.3718078813981265e-05, 1.2181264537503012e-05, 0.9999440908432007]
True Label: 1, Predicted Label: 1, Probabilities: [0.0013498173793777823, 0.9986364245414734, 1.3754033716395497e-05]
True Label: 1, Predicted Label: 1, Probabilities: [0.0009494688

In [6]:

# 저장된 모델과 토크나이저 불러오기
model = AutoModelForSequenceClassification.from_pretrained("./saved_model")
tokenizer = AutoTokenizer.from_pretrained("./saved_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def sentence_predict(sent):
    model.eval()
    tokenized_sent = tokenizer(
        [sent],
        return_tensors="pt",
        max_length=128,
        padding=True,
        truncation=True,
        add_special_tokens=True,
    )
    tokenized_sent = {k: v.to(device) for k, v in tokenized_sent.items()}
    with torch.no_grad():
        outputs = model(**tokenized_sent)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_label = torch.argmax(probabilities, dim=1).item()
    return predicted_label, probabilities.tolist()[0]

corrected_texts = [
    "초등학교는 완전히 끝난거 맞죠?",
    "초등학교 해당되는 강사분들 답변바랍니다",
    "제가알기론 13일날 한번 남은걸로 알고있습니다.!",
    "추가로 스마트팜 조립은 프레임 빼고 다 해놨고 오늘 학생 안와서 5개만 되어있어요.",
    "각 조립에 학생들 이름 붙여놨으니 구별할 수 있습니다.",
    "그냥 그대로 하루에 2차시씩 한거 맞나요?",
    "결과보고서 및 기증문서 대부분은 제가 다 작성해놓았으니 서명받고 사업단에 제출하시면 됩니다",
    "네. 원래 일정상 13일이 마지막 날입니다",
    "안녕하세요, 11월 말일까지 진행했던 해봄학교 교육활동 출석부만 먼저 제출 부탁드립니다~ ",
    "현재 다 완료되지 않은 관계로 활동결과보고서 전체가 아닌 서명부 부탁드립니다. ",
    "지금 현재 본인이 담당하고 있는 클래스 주 강사는 현재까지 '출석부' 날짜 잘 보이게 사진 찍어서 내일 10:00시까지 보내주세요.",
    "출석부 12월 1일까지의 출결사항 및 근무일자 기재 후 필히 보내주시기 바랍니다.",
    "학교 담당자입니다.",
    "내일 회식 참석 전",
    "또는 귀가 전",
    "시발",
    "난 너가 너무 싫어",
    "혐오스러워",
    "진짜 세상 다 안망하네~ 인생 좆같네 진짜",
    "싫다 진자 전부 다 귀찮다 이젠",
    "너가 너무 미워서 정말 시러",
    "죽어",
    "살아",
    "네가 너무 좋아",
    "네가 너무 싫어",
    "행복한 감정",
    "불행한 감정",
    "하... 슬프다 진짜",
]

for text in corrected_texts:
    probabilities = sentence_predict(text)
    print(f"Text: {text}, Probabilities: {probabilities}")

Text: 초등학교는 완전히 끝난거 맞죠?, Probabilities: (2, [0.011781834065914154, 0.007961065508425236, 0.9802570939064026])
Text: 초등학교 해당되는 강사분들 답변바랍니다, Probabilities: (2, [0.00020253067486919463, 4.499606802710332e-05, 0.9997524619102478])
Text: 제가알기론 13일날 한번 남은걸로 알고있습니다.!, Probabilities: (1, [0.003124585608020425, 0.9967798590660095, 9.563820640323684e-05])
Text: 추가로 스마트팜 조립은 프레임 빼고 다 해놨고 오늘 학생 안와서 5개만 되어있어요., Probabilities: (2, [4.1370662074768916e-05, 2.0740928448503837e-05, 0.999937891960144])
Text: 각 조립에 학생들 이름 붙여놨으니 구별할 수 있습니다., Probabilities: (2, [5.492604395840317e-05, 1.3132258573023137e-05, 0.9999319314956665])
Text: 그냥 그대로 하루에 2차시씩 한거 맞나요?, Probabilities: (2, [5.376265835366212e-05, 1.1768489457608666e-05, 0.9999344348907471])
Text: 결과보고서 및 기증문서 대부분은 제가 다 작성해놓았으니 서명받고 사업단에 제출하시면 됩니다, Probabilities: (2, [0.00010052478319266811, 3.162130087730475e-05, 0.9998679161071777])
Text: 네. 원래 일정상 13일이 마지막 날입니다, Probabilities: (2, [0.00011547724716365337, 4.021577115054242e-05, 0.9998443126678467])
